In [1]:
#Import the required packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [2]:
#==================================================================================================================
# Extract information about a single vessel and get the latest header information from http://www.vesseltracking.net
#==================================================================================================================

vessel_name = 'navigator-jorf'
vessel_mmsi = '9796339'


#Extract data from the page

url = 'http://www.vesseltracking.net/ship/'+ vessel_name +'-' + vessel_mmsi
requests.get(url)
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
table_data = soup.find('table', class_ = "table table-striped")

#Extract information about the vessel
table_info = []
for i in table_data.find_all('td'):
    title = i.text.strip()
    table_info.append(title)
    
#Get the name of vessel's characteristics
header = []
for h in range(0, len(table_info), 2):
    header.append(table_info[h])

#Get the characteristics
info  =[]
for i in range(1, len(table_info), 2):
        info.append(table_info[i])
        
#Create a dataframe 
info_arr = np.asarray(info).reshape((1,len(info)))
df = pd.DataFrame(info_arr, columns = header)

#Show the dataset
df

,IMO,Callsign,Flag (Registration),Gross Tonnage,Deadweight (t),Length (m),Beam (m),TEU,Built (year),Builder,Yard
0,9796339,D5NS3,Liberia,24942,28726,179.87,28.4,0,2017,HYUNDAI MIPO DOCKYARD,ULSAN


In [3]:
#==================================================================================================================
# Extract information about a many vessels and get the latest header information from http://www.vesseltracking.net
#==================================================================================================================



#Load the list of vessels for which we want to get the characteristics
#vessels_list = pd.read_excel('path to your list')
#Here is an example of a list of vessels 

example_list = [['HEILAN STAR', 9587984],
['EPIC TRADER', 9551698],
['NORD PEAK', 9566564],
['PAPA JHON', 9555149],
['KM TOKYO',9552812],
['TRANS AUTUMN',9615470]]

vessels_list = pd.DataFrame(example_list, columns = ['VESSEL_NAME','IMO'])

#Change the form of the vessel name 
vessels_list['name'] = vessels_list['VESSEL_NAME'].apply(lambda x: x.lower().replace(" ", "-").replace(".", "-").replace(",", "-"))

#get the list of names and IMO numbers
short_list = vessels_list[['name','IMO']]
vessels_arr = np.array(short_list)

#Function to get a vessel info 
def get_info(name,id):
  td_data = []

  url = 'http://www.vesseltracking.net/ship/{}-{}'.format(name, id)
  requests.get(url)
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'lxml')
  table_data = soup.find('table', class_ = "table table-striped")
  
  for i in table_data.find_all('td'):
    data = i.text.strip()
    td_data.append(data)
    
  info = [td_data[i] for i in range(1, len(td_data), 2)]
  return info

#Function to get the header
def get_header(name,id):
  td_data = []

  url = 'http://www.vesseltracking.net/ship/{}-{}'.format(name, id)
  requests.get(url)
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'lxml')
  table_data = soup.find('table', class_ = "table table-striped")
  
  for i in table_data.find_all('td'):
    data = i.text.strip()
    td_data.append(data)
    
  header = [td_data[i] for i in range(0, len(td_data), 2)]
  return header

#Get all vessels info
vessels_info = []
for name, id in vessels_arr:
    x = get_info(name,id)
    vessels_info.append(x)
    print(name, id)
    
#Get the header

name_, id_ = vessels_arr[0] 
header = get_header (name_,id_)
    
#Create a dataframe from vessels info
vessels_details = pd.DataFrame(columns = header, data=vessels_info)
vessels_details.IMO.astype('int32')
vessels_details = pd.concat([vessels_details,vessels_list], axis=1)

#Show the dataset
vessels_details

heilan-star 9587984
epic-trader 9551698
nord-peak 9566564
papa-jhon 9555149
km-tokyo 9552812
trans-autumn 9615470


,IMO,Callsign,Flag (Registration),Gross Tonnage,Deadweight (t),Length (m),Beam (m),TEU,Built (year),Builder,Yard,VESSEL_NAME,IMO,name
0,9587984,HP5246,Panama,32953,56882,189.99,32.26,0,2011,SHANGHAI SHIPYARD,SHANGHAI,HEILAN STAR,9587984,heilan-star
1,9551698,D5NR9,Liberia,32987,56778,189.99,32.26,0,2012,JIANGSU HANTONG SHIP HEAVY INDUSTRY,TONGZHOU,EPIC TRADER,9551698,epic-trader
2,9566564,9V9236,Singapore,34060,61649,199.98,32.26,270,2011,OSHIMA SHIPBUILDING,SAIKAI,NORD PEAK,9566564,nord-peak
3,9555149,V7TY7,Marshall Islands,33044,56543,189.99,32.26,0,2010,QINGSHAN SHIPYARD,WUHAN,PAPA JHON,9555149,papa-jhon
4,9552812,D5VR7,Liberia,44349,83483,229,32.24,0,2010,SANOYAS HISHINO MEISHO CORPORATION,KURASHIKI,KM TOKYO,9552812,km-tokyo
5,9615470,VRKA7,Hong Kong,33044,56838,190,24.8,1003,2012,DAE SUN SHIPBUILDING & ENGINEERING,BUSAN,TRANS AUTUMN,9615470,trans-autumn
